In [4]:
import os
import json
from agents.general_agent import Agent
from agents.seminal_eval_agent import SeminalEvalAgent
from agents.concept_extraction_agent import ConceptExtractionAgent

In [5]:
# Get the current working directory of the notebook
notebook_dir = os.getcwd()
queries_folder = os.path.join(notebook_dir, "..", "queries")

gpt_agent = Agent(api_key=os.getenv("OPENAI_API_KEY"))

concept_agent = ConceptExtractionAgent(gpt_agent, queries_folder=queries_folder)

try:
    response = concept_agent.extract_concepts("robotics_grasp.json")
    project_title = response["project_title"]
    project_summary = response["project_summary"]
    
    core_concepts = response["core_concepts"]
    specialized_concepts = response["specialized_concepts"]
    fundamental_concepts = response["fundamental_concepts"]
    prerequisites = response["prerequisites"]
except (FileNotFoundError, ValueError) as e:
    print(f"Error: {e}")

Concepts and Metadata:
 {
    "project_title": "Robotic Manipulation in Unstructured Environments",
    "project_summary": "Developing a robotic system that learns to manipulate objects in unstructured environments using reinforcement learning. The system aims to generalize manipulation skills to novel objects and situations.",
    "prerequisites": ["Basic programming skills", "Familiarity with Python", "Basic calculus"],
    "fundamental_concepts": ["Machine Learning Basics", "Robotic Kinematics", "Control Theory"],
    "core_concepts": ["Reinforcement Learning", "Robotics"],
    "specialized_concepts": ["RL for Grasping Strategies", "Visual Input Processing", "Pick-and-Place Tasks"]
}


In [6]:
# Query for survey papers for each core concept
survey_papers = {}
for concept in core_concepts:
    print(f"Querying agent for survey papers on: {concept}")
    
    input_text = f"""
    Provide 6 survey papers on the topic '{concept}' along with their DOI id's, URLs, and other metadata in the following JSON format.
    Do NOT include tick marks or any other formatting. Just ONLY provide the JSON object: 
    {{
        "papers": [
            {{
                "title": "<title>",
                "doi": "<doi>",
                "url": "<url>",
                "abstract": "<abstract>"
            }},
            ...
        ]
    }}
    """
    try:
        gpt_response = gpt_agent.query("You are a helpful assistant.", input_text)
        print(f"Survey papers for {concept}:\n{json.dumps(json.loads(gpt_response), indent=4)}")
        survey_papers[concept] = json.loads(gpt_response)
    except json.JSONDecodeError:
        print(f"Failed to parse GPT-4 response for {concept}. Response: {gpt_response}")
        survey_papers[concept] = []

Querying agent for survey papers on: Reinforcement Learning
Survey papers for Reinforcement Learning:
{
    "papers": [
        {
            "title": "A Survey on Reinforcement Learning: Open Problems and Applications",
            "doi": "10.1016/j.artint.2020.103500",
            "url": "https://www.sciencedirect.com/science/article/pii/S0004370220300725",
            "abstract": "This survey provides a comprehensive overview of the current state of reinforcement learning, highlighting open problems and potential applications."
        },
        {
            "title": "Deep Reinforcement Learning: A Survey",
            "doi": "10.1145/3386252",
            "url": "https://dl.acm.org/doi/10.1145/3386252",
            "abstract": "This paper surveys the field of deep reinforcement learning, discussing its foundations, algorithms, and applications."
        },
        {
            "title": "A Survey of Reinforcement Learning Informed by Natural Language",
            "doi": "10.1613

In [7]:
# Separate dictionaries for seminal paper counts by topic
seminal_paper_counts_by_topic = {}

# Initialize top_references as an empty dictionary
top_references = {}

for concept, papers in survey_papers.items():
    print(f"\nProcessing concept: {concept}")
    top_references[concept] = {}
    seminal_paper_counts_by_topic[concept] = {}

    for paper in papers.get("papers", []):
        title = paper.get("title", "Unknown Title")
        print(f"Retrieving top references for paper: {title}")
        
        input_text = f"""
        For the paper titled '{title}', provide the 5 most seminal works (including papers and textbooks) in the field that are related to this paper and would likely be cited. 
        If you cannot access external databases, respond with 5 hypothetical seminal works based on the paper title in the following example JSON format (not with this content, but with the same structure):
        Do NOT include tick marks or any other formatting. Just ONLY provide the JSON object: 
        {{
            "seminal_works": [
            {{"title": "Seminal Work 1", "year": 1998}},
            {{"title": "Seminal Work 2", "year": 2013}},
            {{"title": "Seminal Work 3", "year": 2015}},
            {{"title": "Seminal Work 4", "year": 2020}},
            {{"title": "Seminal Work 5", "year": 2021}}
            ]
        }}
        """
        try:
            gpt_response = gpt_agent.query("You are a helpful assistant.", input_text)
            references = json.loads(gpt_response).get("seminal_works", [])
            top_references[concept][title] = references
            paper["references"] = references
            
            # Aggregate references into the topic-specific seminal paper counts
            for ref in references:
                ref_title = ref["title"]
                if ref_title in seminal_paper_counts_by_topic[concept]:
                    seminal_paper_counts_by_topic[concept][ref_title] += 1
                else:
                    seminal_paper_counts_by_topic[concept][ref_title] = 1

        except json.JSONDecodeError:
            print(f"Failed to parse GPT-4 response for paper: {title}")
            top_references[concept][title] = []
            paper["references"] = []
    
    # Output the seminal paper counts for the current topic by value
    print(f"\nSeminal Paper Counts for {concept} (sorted by count):")
    sorted_reference = sorted(seminal_paper_counts_by_topic[concept].items(), key=lambda item: item[1], reverse=True)
    for paper_title, count in sorted_reference:
        print(f"{paper_title}: {count}")


Processing concept: Reinforcement Learning
Retrieving top references for paper: A Survey on Reinforcement Learning: Open Problems and Applications
Retrieving top references for paper: Deep Reinforcement Learning: A Survey
Retrieving top references for paper: A Survey of Reinforcement Learning Informed by Natural Language
Retrieving top references for paper: Multi-Agent Reinforcement Learning: A Selective Overview of Theories and Algorithms
Retrieving top references for paper: A Survey on Policy Search for Robotics
Retrieving top references for paper: Reinforcement Learning for Robotics: A Survey

Seminal Paper Counts for Reinforcement Learning (sorted by count):
Reinforcement Learning: An Introduction: 6
Human-level control through deep reinforcement learning: 2
Asynchronous Methods for Deep Reinforcement Learning: 2
Proximal Policy Optimization Algorithms: 2
Playing Atari with Deep Reinforcement Learning: 2
Policy Gradient Methods for Reinforcement Learning with Function Approximatio

In [8]:
# Initialize the evaluation agent
seminal_eval_agent = SeminalEvalAgent(seminal_paper_counts_by_topic, gpt_agent)

# Select papers based on the criteria
selected_papers = seminal_eval_agent.select_papers()

# Output the selected papers for evaluation
print("\nSelected Papers for Evaluation:")
for topic, papers in selected_papers.items():
    print(f"\nTopic: {topic}")
    for paper in papers:
        print(f"- {paper}")

Topic 'Reinforcement Learning': High-count papers selected: ['Reinforcement Learning: An Introduction', 'Human-level control through deep reinforcement learning', 'Asynchronous Methods for Deep Reinforcement Learning', 'Proximal Policy Optimization Algorithms', 'Playing Atari with Deep Reinforcement Learning', 'Policy Gradient Methods for Reinforcement Learning with Function Approximation', 'Trust Region Policy Optimization']
Error assessing relevance for topic 'Reinforcement Learning': could not convert string to float: ''
Topic 'Reinforcement Learning': Final selected papers: ['Reinforcement Learning: An Introduction', 'Human-level control through deep reinforcement learning', 'Asynchronous Methods for Deep Reinforcement Learning', 'Proximal Policy Optimization Algorithms', 'Playing Atari with Deep Reinforcement Learning', 'Policy Gradient Methods for Reinforcement Learning with Function Approximation', 'Trust Region Policy Optimization', 'A Survey of Reinforcement Learning Algorithm

In [9]:
# Prune evaluation papers using specialized topics
def prune_papers_with_specialized_topics(specialized_topics, evaluation_papers):
    """
    Use an LLM agent to filter evaluation papers based on specialized topics.
    """
    pruned_papers = {}
    for topic, papers in evaluation_papers.items():
        pruned_papers[topic] = []
        for paper in papers:
            input_text = f"""
            The project focuses on the following specialized topics: {', '.join(specialized_topics)}.
            Determine if the paper titled '{paper}' is directly relevant either one of these:
            1. general understanding of {topic}
            2. the specialized topics.

            Respond with "yes" if it is relevant or beneficial to understanding the topic in general.
            Otherwise respond with "no".
            """
            try:
                response = gpt_agent.query("You are a helpful assistant.", input_text).strip().lower()
                print(f"Response for paper '{paper}': {response}")
                if response == "yes":
                    print(f"Paper '{paper}' is relevant.")
                    pruned_papers[topic].append(paper)
            except Exception as e:
                print(f"Error processing paper '{paper}': {e}")
    return pruned_papers

# Apply pruning to the selected papers
pruned_selected_papers = prune_papers_with_specialized_topics(specialized_concepts, selected_papers)

Response for paper 'Reinforcement Learning: An Introduction': yes
Paper 'Reinforcement Learning: An Introduction' is relevant.
Response for paper 'Human-level control through deep reinforcement learning': yes
Paper 'Human-level control through deep reinforcement learning' is relevant.
Response for paper 'Asynchronous Methods for Deep Reinforcement Learning': yes
Paper 'Asynchronous Methods for Deep Reinforcement Learning' is relevant.
Response for paper 'Proximal Policy Optimization Algorithms': yes
Paper 'Proximal Policy Optimization Algorithms' is relevant.
Response for paper 'Playing Atari with Deep Reinforcement Learning': yes
Paper 'Playing Atari with Deep Reinforcement Learning' is relevant.
Response for paper 'Policy Gradient Methods for Reinforcement Learning with Function Approximation': yes
Paper 'Policy Gradient Methods for Reinforcement Learning with Function Approximation' is relevant.
Response for paper 'Trust Region Policy Optimization': yes
Paper 'Trust Region Policy Op

In [10]:
from pprint import pprint

pprint(pruned_selected_papers)

# Output the pruned selected papers
print("\nPruned Selected Papers for Evaluation:")
for topic, papers in pruned_selected_papers.items():
    print(f"\nTopic: {topic}")
    for paper in papers:
        print(f"- {paper}")

{'Reinforcement Learning': ['Reinforcement Learning: An Introduction',
                            'Human-level control through deep reinforcement '
                            'learning',
                            'Asynchronous Methods for Deep Reinforcement '
                            'Learning',
                            'Proximal Policy Optimization Algorithms',
                            'Playing Atari with Deep Reinforcement Learning',
                            'Policy Gradient Methods for Reinforcement '
                            'Learning with Function Approximation',
                            'Trust Region Policy Optimization',
                            'A Survey of Reinforcement Learning Algorithms'],
 'Robotics': ['Programming by Demonstration: A Method for Robot Programming',
              'Learning from Demonstration',
              'Robot Learning from Demonstration']}

Pruned Selected Papers for Evaluation:

Topic: Reinforcement Learning
- Reinforcement Le

In [26]:
# Generate foundational topics and resources in two passes
def find_foundational_topics_and_resources(
    pruned_papers, existing_fundamental_concepts
):
    """
    Use an LLM agent to identify foundational topics required for the pruned list of papers
    and recommend resources for each topic in two passes.
    """
    # Initialize the output dictionary
    foundational_topics = {}

    # First pass: Generate foundational topics grouped by paper
    for topic, papers in pruned_papers.items():
        for paper in papers:
            input_text = f"""
            The project involves the following paper: {paper}.
            Identify all foundational topics required to understand this paper.
            Include both the existing foundational topics: {', '.join(existing_fundamental_concepts)}
            and any additional foundational topics not listed. Respond in the following JSON format
            Do NOT include tick marks or any other formatting. Just ONLY provide the JSON object:
            {{
                "foundational_topics": [
                    {{
                        "topic": "<topic>"
                    }},
                    {{
                        "topic": "<topic>"
                    }}
                ]
            }}
            """
            try:
                response = gpt_agent.query("You are a helpful assistant.", input_text)
                print(
                    f"Response for foundational topics for paper '{paper}':", response
                )
                response_data = json.loads(response)
                foundational_topics[paper] = response_data.get(
                    "foundational_topics", []
                )
            except Exception as e:
                print(f"Error generating foundational topics for paper '{paper}': {e}")
                foundational_topics[paper] = []

    # Second pass: Attach resources to each foundational topic
    for paper, topics in foundational_topics.items():
        input_text = f"""
        The following foundational topics have been identified for the paper '{paper}': {', '.join([t['topic'] for t in topics])}.
        For each topic, recommend a research paper, textbook, or resource that provides
        a comprehensive introduction to the topic. Respond in the following JSON format
        Do NOT include tick marks or any other formatting. Just ONLY provide the JSON object:
        {{
            "{paper}": [
                {{
                    "topic": "<topic>",
                    "resource": "<resource title and author or link>"
                }},
                {{
                    "topic": "<topic>",
                    "resource": "<resource title and author or link>"
                }}
            ]
        }}
        """
        try:
            response = gpt_agent.query("You are a helpful assistant.", input_text)
            print(f"Response for resources for paper '{paper}':", response)
            response_data = json.loads(response)
            foundational_topics[paper] = response_data.get(paper, [])
        except Exception as e:
            print(f"Error generating resources for paper '{paper}': {e}")

    return foundational_topics


# Apply the agent to find foundational topics and resources
final_foundational_topics = find_foundational_topics_and_resources(
    pruned_selected_papers, fundamental_concepts
)

# Output the final foundational topics and their resources
print("\nFinal Foundational Topics and Recommended Resources:")
for paper, topics in final_foundational_topics.items():
    print(f"\nPaper: {paper}")
    for topic in topics:
        print(f"- {topic['topic']}: {topic.get('resource', 'No resource available')}")

Response for foundational topics for paper 'Reinforcement Learning: An Introduction': {
    "foundational_topics": [
        {
            "topic": "Machine Learning Basics"
        },
        {
            "topic": "Robotic Kinematics"
        },
        {
            "topic": "Control Theory"
        },
        {
            "topic": "Probability Theory"
        },
        {
            "topic": "Markov Decision Processes"
        },
        {
            "topic": "Dynamic Programming"
        },
        {
            "topic": "Stochastic Processes"
        },
        {
            "topic": "Optimization"
        }
    ]
}
Response for foundational topics for paper 'Human-level control through deep reinforcement learning': {
    "foundational_topics": [
        {
            "topic": "Machine Learning Basics"
        },
        {
            "topic": "Robotic Kinematics"
        },
        {
            "topic": "Control Theory"
        },
        {
            "topic": "Deep Learnin

In [24]:
pprint(final_foundational_topics)

# Output the final foundational topics and their resources
print("\nFinal Foundational Topics and Recommended Resources:")
for topic, resource in final_foundational_topics.items():
    pprint(f"- {topic}: {resource}")

{'A Survey of Reinforcement Learning Algorithms': [{'resource': 'Pattern '
                                                                'Recognition '
                                                                'and Machine '
                                                                'Learning by '
                                                                'Christopher '
                                                                'M. Bishop',
                                                    'topic': 'Machine Learning '
                                                             'Basics'},
                                                   {'resource': 'Introduction '
                                                                'to Robotics: '
                                                                'Mechanics and '
                                                                'Control by '
                                                            

In [27]:
from utils.tree_builder import build_tree_graph
from utils.visualization import visualize

# project_title = "Title of the Project"
# core_concepts = ["Reinforcement Learning"]
# pruned_selected_papers = {
#     "Reinforcement Learning": [
#         "Reinforcement Learning: An Introduction",
#         "Human-level control through deep reinforcement learning",
#     ]
# }
# foundational_topics = {
#     "Reinforcement Learning: An Introduction": [
#         {
#             "topic": "Machine Learning Basics",
#             "resource": "Pattern Recognition and Machine Learning by Christopher M. Bishop",
#         },
#         {
#             "topic": "Neural Networks",
#             "resource": "Deep Learning by Ian Goodfellow, Yoshua Bengio, and Aaron Courville",
#         },
#     ],
#     "Human-level control through deep reinforcement learning": [
#         {
#             "topic": "Machine Learning Basics",
#             "resource": "Pattern Recognition and Machine Learning by Christopher M. Bishop",
#         }
#     ],
# }

paper_metadata = {
    "Reinforcement Learning: An Introduction": {
        "authors": "Richard S. Sutton, Andrew G. Barto",
        "year": 1998,
        "doi": "10.1109/9780470544787",
        "url": "https://example.com/reinforcement-learning-introduction",
    },
    "Human-level control through deep reinforcement learning": {
        "authors": "Volodymyr Mnih, Koray Kavukcuoglu, David Silver, et al.",
        "year": 2015,
        "doi": "10.1038/nature14236",
        "url": "https://example.com/human-level-control",
    },
}

G = build_tree_graph(
    project_title,
    core_concepts,
    pruned_selected_papers,
    final_foundational_topics,
    paper_metadata,
)

# Visualize with pyvis
visualize(G)

Machine Learning Basics
Pattern Recognition and Machine Learning by Christopher M. Bishop
Robotic Kinematics
Introduction to Robotics: Mechanics and Control by John J. Craig
Control Theory
Feedback Control of Dynamic Systems by Gene F. Franklin, J. Da Powell, and Abbas Emami-Naeini
Probability Theory
A First Course in Probability by Sheldon Ross
Markov Decision Processes
Markov Decision Processes: Discrete Stochastic Dynamic Programming by Martin L. Puterman
Dynamic Programming
Dynamic Programming and Optimal Control by Dimitri P. Bertsekas
Stochastic Processes
Introduction to Stochastic Processes by Gregory F. Lawler
Optimization
Convex Optimization by Stephen Boyd and Lieven Vandenberghe
Machine Learning Basics
Pattern Recognition and Machine Learning by Christopher M. Bishop
Robotic Kinematics
Introduction to Robotics: Mechanics and Control by John J. Craig
Control Theory
Feedback Control of Dynamic Systems by Gene F. Franklin, J. Da Powell, and Abbas Emami-Naeini
Deep Learning
Deep